<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>

# LangGraph Router Pattern: Intent-Based Agent Routing
In this tutorial, we'll explore the Router pattern using LangGraph, a framework for building dynamic, stateful LLM applications. We’ll build a smart support assistant that can route customer queries to the appropriate specialized agent—Billing, Tech Support, or General Info—based on the user’s intent.

LangGraph enables this by letting us define a structured graph of logic, where a router node classifies the input, and conditional edges forward it to the correct sub-agent. Each agent uses local context (e.g., invoice history, troubleshooting tips) to respond intelligently.

We also trace this application using Phoenix, which gives us complete visibility into routing decisions, tool usage, and model interactions. This is helpful for debugging routing accuracy and understanding how the graph executes end-to-end.

In [ ]:
!pip install langgraph langchain langchain_community "arize-phoenix==9.0.1" arize-phoenix-otel openinference-instrumentation-langchain

In [ ]:
!pip install langchain_openai

In [ ]:
import getpass
import os

from langgraph.graph import END, START, StateGraph

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

# Configure Phoenix Tracing

Make sure you go to https://app.phoenix.arize.com/ and generate an API key. This will allow you to trace your Langgraph application with Phoenix.

In [ ]:
PHOENIX_API_KEY = getpass.getpass("Phoenix API Key:")
os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"

In [ ]:
from phoenix.otel import register

tracer_provider = register(project_name="Router", auto_instrument=True)

In [ ]:
from langchain_openai import ChatOpenAI

# LLM

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

In [ ]:
from typing import Literal, TypedDict

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.pydantic_v1 import BaseModel, Field

# Router
We define a Pydantic schema to structure the router’s output. This schema ensures the LLM returns one of three valid routing targets: "billing", "tech_support", or "general_info".
We then wrap our LLM with with_structured_output, allowing LangGraph to enforce this structured response during routing.

In [ ]:
class Route(BaseModel):
    step: Literal["billing", "tech_support", "general_info"] = Field(
        description="Classify the support request"
    )


router = llm.with_structured_output(Route)

# Defining Graph State
This state schema captures the lifecycle of a routed request. It stores the original user input (input), the classification decision made by the router (decision), and the final response generated by the appropriate support handler (output). Each node in the graph will read from and write to this shared state.

In [ ]:
class State(TypedDict):
    input: str
    decision: str
    output: str

# Support Agent Nodes: Specialized Response Handlers
This section defines three specialized LLM-powered agents, each responsible for handling a different category of user queries:

**Billing Agent**: Uses user billing history and a billing policy context to respond to invoice or refund-related questions.

**Tech Support Agent**: Answers common troubleshooting queries using a predefined support knowledge base.

**General Info Agent**: Responds to account, subscription, and policy-related questions using general FAQs.

Each agent reads the input field from the graph's state and returns a generated output tailored to its domain. These agents form the execution endpoints of the router graph.

In [ ]:
# Billing Agent
billing_history_data = [
    {"invoice_id": "INV001", "date": "2024-11-01", "amount": "$29.99"},
    {"invoice_id": "INV002", "date": "2024-12-01", "amount": "$29.99"},
    {"invoice_id": "INV003", "date": "2025-01-01", "amount": "$39.99"},
]

billing_general_context = (
    "Billing inquiries may include refunds, invoices, plan upgrades, or charges. "
    "Our system charges users monthly based on their plan. Refunds are processed within 5–7 business days."
)


def billing_agent(state: State):
    user_input = state["input"]
    invoice_summary = "\n".join(
        f"• Invoice {item['invoice_id']} on {item['date']} for {item['amount']}"
        for item in billing_history_data
    )
    prompt = (
        f"You are a helpful billing assistant.\n"
        f"Here is the user's billing history:\n{invoice_summary}\n\n"
        f"General billing context:\n{billing_general_context}\n\n"
        f"User query:\n{user_input}"
    )
    result = llm.invoke(prompt)
    return {"output": result.content}


# Tech Support Agent
tech_support_kb = (
    "Common issues include login errors, app crashes, and network connectivity. "
    "To fix login errors, check your email and reset your password. "
    "For crashes, try reinstalling the app. If your connection is unstable, restart your router."
)


def tech_support_agent(state: State):
    prompt = (
        "You are a tech support assistant. Use the knowledge base below to answer the user's question.\n\n"
        f"Knowledge Base:\n{tech_support_kb}\n\n"
        f"User query:\n{state['input']}"
    )
    result = llm.invoke(prompt)
    return {"output": result.content}


# General Info Agent
general_info_kb = (
    "We offer 3 subscription plans: Basic, Pro, and Enterprise. "
    "Support is available 24/7. You can cancel your subscription any time from the account settings page."
)


def general_info_agent(state: State):
    prompt = (
        "You are a general info assistant. Use the knowledge base below to answer the user's question.\n\n"
        f"Knowledge Base:\n{general_info_kb}\n\n"
        f"User query:\n{state['input']}"
    )
    result = llm.invoke(prompt)
    return {"output": result.content}

# Intent Classification & Routing Logic
This section introduces the decision-making logic that powers the routing mechanism:

**classify_intent Node**: Uses an LLM wrapped with a structured output schema to classify the user's query into one of three support categories — billing, tech_support, or general_info. The result is stored in the state's decision key.

**route_to_agent Function**: A conditional router that examines the classification result and sends the request to the corresponding specialized agent node. This allows LangGraph to dynamically direct queries to the most relevant response module.

In [ ]:
def classify_intent(state: State):
    decision = router.invoke(
        [
            SystemMessage(
                content="Classify this support request as billing, tech_support, or general_info."
            ),
            HumanMessage(content=state["input"]),
        ]
    )
    return {"decision": decision.step}


def route_to_agent(state: State):
    if state["decision"] == "billing":
        return "billing_agent"
    elif state["decision"] == "tech_support":
        return "tech_support_agent"
    elif state["decision"] == "general_info":
        return "general_info_agent"

# Building the Routing Graph
Here, we construct the complete LangGraph workflow by registering the nodes and defining their connections:

**Node Registration**: All agents (billing_agent, tech_support_agent, general_info_agent) and the classify_intent node are added to the graph.

**Conditional Routing**: After the graph starts at classify_intent, it uses the output of route_to_agent to forward the query to the appropriate agent node based on the classified intent.

**Terminal Edges**: Each agent node directly leads to END, finalizing the workflow after responding to the query.

This design reflects a typical customer support router architecture, enabling modular, extensible handling of diverse query types.

In [ ]:
builder = StateGraph(State)

builder.add_node("classify_intent", classify_intent)
builder.add_node("billing_agent", billing_agent)
builder.add_node("tech_support_agent", tech_support_agent)
builder.add_node("general_info_agent", general_info_agent)

builder.add_edge(START, "classify_intent")

builder.add_conditional_edges(
    "classify_intent",
    route_to_agent,
    {
        "billing_agent": "billing_agent",
        "tech_support_agent": "tech_support_agent",
        "general_info_agent": "general_info_agent",
    },
)

builder.add_edge("billing_agent", END)
builder.add_edge("tech_support_agent", END)
builder.add_edge("general_info_agent", END)

workflow = builder.compile()

# Let's run some queries:

In [ ]:
queries = [
    "Why was I charged $39.99 this month?",  # billing
    "The app keeps crashing when I open it.",  # tech support
    "Can I cancel my subscription anytime?",  # general info
    "Can you show me all my past invoices?",  # billing
    "How do I fix login issues with my account?",  # tech support
]

# Run each query through the router
for i, query in enumerate(queries, start=1):
    print(f"\n--- Query {i} ---")
    state = workflow.invoke({"input": query})
    print(f"Input: {query}")
    print(f"Response:\n{state['output']}")

# Make sure to view your traces in Phoenix!

# Evals

In this section we will add a simple eval that evaluates whether the router chose the correct subagent. We will use the tool calling eval template from Phoenix, as routing to sub agents mimics the process of picking a tool to call.

## Get all root spans

In [ ]:
import phoenix as px
from phoenix.evals import (
    TOOL_CALLING_PROMPT_RAILS_MAP,
    TOOL_CALLING_PROMPT_TEMPLATE,
    OpenAIModel,
    llm_classify,
)

df = px.Client().get_spans_dataframe("name == 'LangGraph'", project_name="Router")
df

In [ ]:
df.rename(columns={"attributes.output.value": "tool_call"}, inplace=True)
df.rename(columns={"attributes.input.value": "question"}, inplace=True)

In [ ]:
df_clean = df[["context.span_id", "question", "tool_call"]]
df_clean

## Add our subagent definitions as tools

Each column must contain information regarding the tools that could have been chosen from. Since our "tools" in this case are our subagents, we pass in the definitions (descriptions) of our subagents.

In [ ]:
subagent_definitions = """
 billing: Uses user billing history and a billing policy context to respond to invoice or refund-related questions.
tech_support: Answers common troubleshooting queries using a predefined support knowledge base.
general_info: Responds to account, subscription, and policy-related questions using general FAQs.
"""
df_clean["tool_definitions"] = subagent_definitions

In [ ]:
df_clean

# Extracting the subagent that was chosen.

In [ ]:
import re


def extract_decision(x):
    """Extracts the decision value from a string using regex.

    Handles cases where the regex search returns None.
    """
    if x is None:
        return None
    match = re.search(r'"decision"\s*:\s*"([^"]+)"', x)
    if match:
        return match.group(1)
    else:
        return None  # Or any default value you prefer


df_clean["tool_call"] = df_clean["tool_call"].apply(extract_decision)
df_clean

In [ ]:
df_clean.to_csv("router_evals.csv", index=False)

# Running Evals

In [ ]:
eval_model = OpenAIModel(model="gpt-4o")

In [ ]:
rails = list(TOOL_CALLING_PROMPT_RAILS_MAP.values())

response_classifications = llm_classify(
    dataframe=df_clean,
    template=TOOL_CALLING_PROMPT_TEMPLATE,
    model=eval_model,
    rails=rails,
    provide_explanation=True,
)
response_classifications["score"] = (response_classifications["label"] == "correct").astype(int)

In [ ]:
response_classifications

# Exporting Evals to Phoenix!

In [ ]:
response_classifications.drop(
    columns=["exceptions", "execution_status", "execution_seconds"], inplace=True
)

In [ ]:
response_classifications.to_parquet("router_evals.parquet")

In [ ]:
from phoenix.trace import SpanEvaluations

df = response_classifications.copy()
df.index.name = "span_id"
px.Client().log_evaluations(
    SpanEvaluations(
        eval_name="Routing Eval",
        dataframe=df,
    )
)